<a href="https://colab.research.google.com/github/drustantmetar/DataFrame1/blob/main/Spark_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

spark data frame
**bold text**

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [8]:
#sc: SparkContext= SparkSession\
spark = SparkSession\
.builder\
.appName("Spark-SQL")\
.master("local")\
.getOrCreate()

In [31]:
moviesdf=spark.read.option("header","true").option("inferschema","true").format("csv").load("/content/movies.csv")
ratingsdf=spark.read.format("csv").option("header","true").option("inferschema","true").load("/content/ratings.csv")

In [33]:
moviesdf.printSchema()
ratingsdf.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [34]:
moviesdf.select("movieId","title").show(5)

+-------+--------------------+
|movieId|               title|
+-------+--------------------+
|      1|    Toy Story (1995)|
|      2|      Jumanji (1995)|
|      3|Grumpier Old Men ...|
|      4|Waiting to Exhale...|
|      5|Father of the Bri...|
+-------+--------------------+
only showing top 5 rows



In [35]:
ratingsdf.filter("rating<3.0").show(10)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     2|    891|   2.0|974820969|
|     2|   1969|   2.0|974820969|
|     2|   1970|   2.0|974820969|
|     2|   1971|   2.0|974820943|
|     2|   1972|   2.0|974820969|
|     2|   1986|   2.0|974820969|
|     2|   2291|   2.0|974820748|
|     2|   2791|   2.0|974820659|
|     3|    173|   2.0|945176099|
|     3|    512|   2.0|945176005|
+------+-------+------+---------+
only showing top 10 rows



In [38]:
ratingsdf.groupBy("rating").count().orderBy("rating").show()

+------+-------+
|rating|  count|
+------+-------+
|   0.5| 162322|
|   1.0| 455078|
|   1.5| 186839|
|   2.0| 960915|
|   2.5| 595149|
|   3.0|2871144|
|   3.5|1479467|
|   4.0|3723912|
|   4.5|1030401|
|   5.0|1941520|
+------+-------+



In [40]:
ratingsdf.groupBy("rating").count().orderBy(col("rating").desc()).show()

+------+-------+
|rating|  count|
+------+-------+
|   5.0|1941520|
|   4.5|1030401|
|   4.0|3723912|
|   3.5|1479467|
|   3.0|2871144|
|   2.5| 595149|
|   2.0| 960915|
|   1.5| 186839|
|   1.0| 455078|
|   0.5| 162322|
+------+-------+

